In [ ]:
import pandas as pd
import numpy as np

import holidays
from collections import Counter
#from datetime import timedelta, date
import statsmodels
from sklearn.preprocessing import LabelEncoder
from statsmodels.formula.api import ols
from scipy import stats
import seaborn as sns

from tqdm import tqdm
import itertools

from scipy.spatial.distance import euclidean
import time
#%matplotlib inline

import lightgbm as lgb


In [ ]:
x_train = pd.read_csv('/Users/s0c02nj/Desktop/merch_ts/mode_imp_all.csv')
x_test =  pd.read_csv('/Users/s0c02nj/Desktop/merch_ts/test_data_date.csv')
#df_test = pd.read_excel('/Users/s0c02nj/Desktop/merch_ts/test_data.xlsx')

#### To check if new stores are present or not in test

In [ ]:
x_train.head()

In [ ]:
x_train['ind'] = 'tr'

In [ ]:
x_test['ind'] = 'te'

In [ ]:
df_train = x_train[['mds_fam_id','dc_nbr','date','dept_nbr','vendor_nm',
                   'unit_cost','corp_retail_price','ind']]

y_train = x_train[['actual_packs_order']]


df_test = x_test[['mds_fam_id','dc_nbr','date','dept_nbr','vendor_nm',
                   'unit_cost','corp_retail_price','ind']]

In [ ]:
df_train.head()

In [ ]:
df_test.head()

In [ ]:
df_total = pd.concat([df_train,df_test])

In [ ]:
df_total.head()

In [ ]:
df_total['date'] = pd.to_datetime(df_total['date'])

df_total['week'] =  df_total['date'].dt.week
df_total['year'] =  df_total['date'].dt.year
df_total['month'] = df_total['date'].dt.month


In [ ]:
df_total['mds_dc'] = df_total['mds_fam_id'].astype(str) + '_' + df_total['dc_nbr'].astype(str)
df_total['dept_dc'] = df_total['dc_nbr'].astype(str) + '_' + df_total['dept_nbr'].astype(str)
df_total['dept_mds'] = df_total['mds_fam_id'].astype(str) + '_' + df_total['dept_nbr'].astype(str)

In [ ]:
#df_train['dept_week'] = df_train['dept_nbr'].astype(str) + '_' + df_train['week'].astype(str)
#df_train['dept_month'] = df_train['dept_nbr'].astype(str) + '_' + df_train['month'].astype(str)
df_total['mds_week'] = df_total['mds_fam_id'].astype(str) + '_' + df_total['week'].astype(str)

In [ ]:
y_p1 = list(y_train['actual_packs_order'].values)
y_p2 = [0]*x_test.shape[0]
y_p = y_p1 + y_p2

df_total['order'] = y_p

In [ ]:
df_train1 = df_total[df_total['ind'] == 'tr']

In [ ]:
cat_cols = ['mds_fam_id',
             'dc_nbr',
             'dept_nbr',
             'week',
             'year',
             'month',
             'mds_dc',
             'dept_dc',
             'dept_mds',
             'mds_week']

In [ ]:
for i,col in tqdm(enumerate(cat_cols)):
    med = df_train1.groupby(col)['order'].median()
    df_total[str(col)+'count'+'1'] = df_total[col].map(med)

In [ ]:
df_total1 = df_total.sort_values(by=['mds_dc','date'])

In [ ]:
df_total1.columns

In [ ]:
df_total1.index = range(0,len(df_total1))

In [ ]:
# imp_cols = ['mds_fam_id','dc_nbr','date','dept_nbr','corp_retail_price',
#             'mds_dc','dept_dc','dept_mds','mds_week','order','ind']

In [ ]:
unique_md_dc = list(df_total1['mds_dc'].unique())

In [ ]:
def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
    """
    Frame a time series as a supervised learning dataset.
    Arguments:
        data: Sequence of observations as a list or NumPy array.
        n_in: Number of lag observations as input (X).
        n_out: Number of observations as output (y).
        dropnan:Boolean whether or not to drop rows with NaN values.
    Returns:
        Pandas DataFrame of series framed for supervised learning.
    """
    n_vars = 1 if type(data) is list else data.shape[1]
    df = pd.DataFrame(data)
    cols, names = list(), list()
    # input sequence (t-n, ... t-1)
    for i in range(n_in, 0, -1):
        cols.append(df.shift(i))
        names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]
    # forecast sequence (t, t+1, ... t+n)
    for i in range(0, n_out):
        cols.append(df.shift(-i))
        if i == 0:
            names += [('var%d(t)' % (j+1)) for j in range(n_vars)]
        else:
            names += [('var%d(t+%d)' % (j+1, i)) for j in range(n_vars)]
    # put it all together
    agg = pd.concat(cols, axis=1)
    agg.columns = names
    # drop rows with NaN values
    if dropnan:
        agg.dropna(inplace=True)
    return agg

In [ ]:
df_combined = pd.DataFrame()


x_t1 = df_total1

for u_id in tqdm(unique_md_dc) :
    
    #Groupby
    df_temp = x_t1[x_t1['mds_dc'] == u_id].sort_values(by='date')
    #df_temp = df_temp.drop(['date'],axis=1)
    #print (df_temp.head())
    
    #Adding Lag
    df_temp = series_to_supervised(df_temp, n_in=13, n_out=1, dropnan = True)
    df_temp.index = range(0,len(df_temp))
    
    #Appending
    df_combined = pd.concat([df_combined,df_temp],axis=0)
    
    

In [ ]:
df_combined.head()

In [ ]:
df_total1.head()

In [ ]:
df_combined1 = df_combined[['var10(t-13)','var10(t-12)','var10(t-11)','var10(t-10)','var10(t-9)',
                            'var10(t-8)','var10(t-7)','var10(t-6)','var10(t-5)','var10(t-4)',
                            'var10(t-3)','var10(t-2)','var10(t-1)', 
                            'var1(t)','var2(t)', 'var3(t)', 'var4(t)', 'var5(t)', 
                            'var6(t)', 'var7(t)','var8(t)','var9(t)','var10(t)','var11(t)']]



In [ ]:
new_cols = []
for i in range(0,len(list(df_combined1.columns))):
    new_cols.append('x'+ str(i) )

In [ ]:
df_combined1.columns = new_cols

In [ ]:
df_combined1.head()

In [ ]:
df_train_v1 = df_combined1[df_combined1['x23'] == 'tr']

In [ ]:
df_test_v1 =  df_combined1[df_combined1['x23'] == 'te']

Validation Split

In [ ]:
df_comb_tr = pd.DataFrame()
df_comb_val = pd.DataFrame()


x_t1 = df_train_v1

for u_id in tqdm(unique_md_dc) :
    
    #Groupby
    df_temp1 = x_t1[x_t1['x8'] == u_id].sort_values(by='x5')
    df_temp1 = df_temp1.drop(['x5','x13'],axis=1)
    
    df_temp1.index = range(0,len(df_temp1))
    
    #Train-test split
    df_temp_tr = df_temp1[0:int(df_temp1.shape[0]*0.99)]
    df_temp_val = df_temp1[int(df_temp1.shape[0]*0.01):]
    #print (df_temp.head())
    
    #Appending
    df_comb_tr =  pd.concat([df_comb_tr,df_temp_tr],axis=0)
    df_comb_val = pd.concat([df_comb_val,df_temp_val],axis=0)

In [ ]:
df_comb_val.head()

In [ ]:
x_train = df_comb_tr.drop(['x12'],axis=1)
y_train = df_comb_tr['x12']

x_val = df_comb_val.drop(['x12'],axis=1)
y_val = df_comb_val['x12']

In [ ]:
le = LabelEncoder()
le_cols = ['x8','x9','x10','x11']

for col in tqdm(le_cols):
    le = LabelEncoder()
    le = le.fit(list(set(x_train[col])) + list(set(x_val[col])) + list(set(df_test_v1[col])))
    
    x_train[col] = le.transform(x_train[col])
    x_val[col] = le.transform(x_val[col])
    df_test_v1[col] = le.transform(df_test_v1[col])
    

In [ ]:
# dummy_model =  ols(formula='x12 ~ C(x0)+ C(x1) + C(x2)'
#                   '+ C(x3)+C(x4)+C(x6)'
#                   '+ C(x7)+C(x8)+C(x9)+C(x10)',
#                    data= df_comb_tr).fit()

In [ ]:
params = {
    'boosting_type': 'gbdt',
    'objective': 'regression',
    'metric': 'rmse',
    'max_depth': 6, 
    'learning_rate': 0.1,
    'verbose': 1, 
    }

In [ ]:
n_estimators = 1000

In [ ]:
d_train = lgb.Dataset(x_train, label = y_train)
d_valid = lgb.Dataset(x_val, label = y_val)

In [ ]:
model = lgb.train(params,
                  d_train, 
                  n_estimators,
                  valid_sets = [d_train, d_valid],
                  early_stopping_rounds=500,
                  verbose_eval=1)


In [ ]:
#preds = model.predict(x_valid)

In [ ]:
y_pred = model.predict(x_val, num_iteration=model.best_iteration)

In [ ]:
from sklearn.metrics import r2_score
r2 = r2_score(y_val, y_pred)

In [ ]:
r2

In [ ]:
a = df_test_v1['x5'].iloc[0]
a

Test_Prediction

In [ ]:
df_test_v1 = df_test_v1.sort_values(by=['x8','x5'],ascending=True)

In [ ]:
df_test_v1.index = range(0,len(df_test_v1))

In [ ]:
test_id = list(set(df_test_v1['x8']))

In [ ]:
df_test_v1['x5'] = df_test_v1['x5'].apply(lambda x : x.strftime("%Y-%m-%d (%H:%M:%S.%f)")[0:10])

In [ ]:
test_id

In [ ]:
dict_pred = {}



for id_md in tqdm(test_id):
    
    temp1 = df_test_v1[df_test_v1['x8'] == id_md ]
    list_dt = sorted(list(temp1['x5']))
    
    
    for i in range(0,len(list_dt)):
        
        if i == 0:
            temp2 = temp1[temp1['x5'] == list_dt[i]].drop(['x5','x12','x13'],axis=1)
            pred = model.predict(temp2)[0]
            dict_pred[(id_md,list_dt[i])] = pred
        
        elif i ==1 :
            temp2 = temp1[temp1['x5'] == list_dt[i]].drop(['x5','x12','x13'],axis=1)
            temp2['x2'] = dict_pred[(id_md,list_dt[i-1])]
            pred = model.predict(temp2)[0]
            dict_pred[(id_md,list_dt[i])] = pred
        
        elif i ==2 :
            temp2 = temp1[temp1['x5'] == list_dt[i]].drop(['x5','x12','x13'],axis=1)
            temp2['x2'] = dict_pred[(id_md,list_dt[i-1])]
            temp2['x1'] = dict_pred[(id_md,list_dt[i-2])]
            pred = model.predict(temp2)[0]
            dict_pred[(id_md,list_dt[i])] = pred
        
        elif i >2 :
            temp2 = temp1[temp1['x5'] == list_dt[i]].drop(['x5','x12','x13'],axis=1)
            temp2['x2'] = dict_pred[(id_md,list_dt[i-1])]
            temp2['x1'] = dict_pred[(id_md,list_dt[i-2])]
            temp2['x0'] = dict_pred[(id_md,list_dt[i-3])]
            pred = model.predict(temp2)[0]
            dict_pred[(id_md,list_dt[i])] = pred
        


In [ ]:
#temp1 = df_test_v1[df_test_v1['x8'] == id_md ]
#list_dt = sorted(list(temp1[temp1['x5']]))

In [ ]:
test_pred = []

for i in tqdm(range(0,len(df_test_v1))):
    out = dict_pred[(df_test_v1['x8'].iloc[i],df_test_v1['x5'].iloc[i])]
    test_pred.append(out)

In [ ]:
df_test_v1['output'] = test_pred

In [ ]:
le1= LabelEncoder()
le1.fit(list(df_combined1['x8'].unique()))

In [ ]:
#df_test_v1

In [ ]:
df_test_v1['x8'] = le1.inverse_transform(df_test_v1['x8'])

In [ ]:
df_test_v1

In [ ]:
dict_pred2 = {}

for i in tqdm(range(0,len(df_test_v1))):
    id1 = df_test_v1['x8'].iloc[i]
    dt = df_test_v1['x5'].iloc[i]
    dict_pred2[(id1,dt)] = df_test_v1['output'].iloc[i]

In [1]:
final_test = pd.read_csv('/Users/s0c02nj/Downloads/validation_data.csv')

NameError: name 'pd' is not defined

In [ ]:
final_test['cal_date'].iloc[0]

In [ ]:
final_test

In [ ]:
p_temp = []
for i in range(0,len(final_test)):
    
    id1 = final_test['mds_fam_id_dc_nbr'].iloc[i]
    dt = final_test['cal_date'].iloc[i]
    
    p_temp.append(dict_pred2[(id1,dt)])

In [ ]:
final_test['pred'] = p_temp

In [ ]:
#final_test.to_csv('/Users/s0c02nj/Desktop/merch_ts/final_test.csv',index = False)

In [ ]:
p = sorted(list(df_test_v1[(df_test_v1['x8'] == 0)]['x5']))[0]

In [ ]:
a1 = df_test_v1[(df_test_v1['x8'] == 0)&(df_test_v1['x5'] == '2019-08-10')].drop(['x5','x13','x12'],axis=1)
a1

In [ ]:
a1['x2'] = 10
a1['x1'] = 5

In [ ]:
a1

In [ ]:
# p=pd.DataFrame(lag_y)
# q =pd.DataFrame(lag_x)
df_train.head()

In [ ]:
df_train['date'] = pd.to_datetime(df_train['date'])

df_train['week'] =  df_train['date'].dt.week
df_train['year'] =  df_train['date'].dt.year
df_train['month'] = df_train['date'].dt.month


In [ ]:
# Select country
us_holidays = holidays.US()

df_train['holiday'] = df_train['date'].apply(lambda x: int(x in us_holidays))

In [ ]:
df_train['mds_dc'] = df_train['mds_fam_id'].astype(str) + '_' + df_train['dc_nbr'].astype(str)
df_train['dept_dc'] = df_train['dc_nbr'].astype(str) + '_' + df_train['dept_nbr'].astype(str)
df_train['dept_mds'] = df_train['mds_fam_id'].astype(str) + '_' + df_train['dept_nbr'].astype(str)

In [ ]:
df_train['dept_week'] = df_train['dept_nbr'].astype(str) + '_' + df_train['week'].astype(str)
#df_train['dept_month'] = df_train['dept_nbr'].astype(str) + '_' + df_train['month'].astype(str)
df_train['mds_week'] = df_train['mds_fam_id'].astype(str) + '_' + df_train['week'].astype(str)
#df_train['dept_month'] = df_train['mds_fam_id'].astype(str) + '_' + df_train['month'].astype(str)

In [ ]:
df_train['order'] = y_train

In [ ]:
unique_md_dc = list(x_t1['mds_dc'].unique())
unique_md_dc[0]

In [ ]:
cols = list(x_t1.columns)
# cols = cols + ['date']
#list(x_t1.columns)

#cols = ['mds_fam_id','dc_nbr','dept_nbr','mds_dc','dept_dc','dept_mds','dept_week','mds_week','order','date']

In [ ]:
# df_combined = pd.DataFrame()
# x_t1 = df_train[cols]

# for u_id in tqdm(unique_md_dc) :
    
#     #Groupby
#     df_temp = x_t1[x_t1['mds_dc'] == u_id].sort_values(by='date')
#     df_temp = df_temp.drop(['date'],axis=1)
#     #print (df_temp.head())
    
#     #Adding Lag
#     df_temp = series_to_supervised(df_temp, n_in=3, n_out=1, dropnan = True)
#     df_temp.index = range(0,len(df_temp))
    
#     #Appending
#     df_combined = pd.concat([df_combined,df_temp],axis=0)
    
    

In [ ]:
df_combined.head()

In [ ]:
df_combined.columns

In [ ]:
df_combined1 = df_combined[['var9(t-3)','var9(t-2)','var9(t-1)', 'var1(t)','var2(t)', 'var3(t)', 'var4(t)', 'var5(t)', 
                            'var6(t)', 'var7(t)','var8(t)','var9(t)']]

In [ ]:
#x_t1['order'] = y_train
list(df_combined1.columns)

In [ ]:
new_cols = []
for i in range(0,len(list(df_combined1.columns))):
    new_cols.append('x'+ str(i) )

In [ ]:
new_cols

In [ ]:
df_combined1.columns = new_cols

In [ ]:
dummy_model = ols(formula='x11 ~ C(x0)+C(x1)+C(x2)'
                  '+ C(x3)+C(x4)+C(x5)'
                  '+ C(x6)+C(x7)+C(x8)+C(x9)+C(x10)',
                   data= df_combined1).fit()

In [ ]:
dummy_model.summary()

In [ ]:
#sns.distplot(dummy_model.resid)


#### Transition Matrix based approach

In [ ]:
df1 = df_train[(df_train['mds_fam_id'] == 105899148) & (df_train['dc_nbr'] == 6006)]

In [ ]:
df1 = df1.sort_values(by='wm_year_wk_nbr')

In [ ]:
list_states = list(df1['actual_packs_order'])
states = list(set(list_states))

In [ ]:
le = LabelEncoder()
states_enc = le.fit_transform(states)+1

In [ ]:
#list_states_enc = le.transform(list_states)+1

In [ ]:
#le.inverse_transform(states_enc-1)
#list_states_enc

In [ ]:
def step_transition_matrix(transitions,k):
    
    n = len(np.unique(transitions)) #number of states

    M = [[0]*n for _ in range(n)]

    for (i,j) in zip(transitions,transitions[k:]):
        M[i-1][j-1] += 1

    #now convert to probabilities:
    for row in M:
        s = sum(row)
        if s > 0:
            row[:] = [f/s for f in row]
    return M


Transition Matrix

In [ ]:
A_lag1 = np.array(step_transition_matrix(states_enc,1))

In [ ]:
#step_transition_matrix(x,3)
#A_lag1
A_lag1.shape

No new stores are present

#### Time Series Analysis on Store Data ------> Temporal Part of the Data

In [ ]:
unique_dc = list(df_train['dc_nbr'].unique())

In [ ]:
unique_mds = list(df_train['mds_fam_id'].unique())

In [ ]:
#Store1_data_analysis
#df_s1 = df_train[(df_train['dc_nbr'] == 6042) & (df_train['mds_fam_id'] == 72389674)]
df_s1 = df_train

In [ ]:
df_s1.head()

In [ ]:
#df['date_in'] = pd.to_datetime(df['date_in'])
#df_sorted = df_s1.sort_values(by=['wm_year_wk_nbr'])
df_sorted = df_s1

In [ ]:
df_sorted.head(2)

In [ ]:
#df_sorted = df_sorted.set_index('wm_year_wk_nbr')
plt.plot(df_sorted['actual_packs_order'])
#plt.xticks(rotation=70)


In [ ]:
#Time Series Testing
timeseries = df_sorted['actual_packs_order'].values
adfTest = adfuller(timeseries, autolag='AIC')
adfTest[1]

In [ ]:
durbin_watson(timeseries)

+ The Durbin Watson test reports a test statistic, with a value from 0 to 4, where:
+ 2 is no autocorrelation.
+ 0 to <2 is positive autocorrelation (common in time series data).

+ Durbin Watson Test of Serial Correlation indicates that their is serial correlation present in the observations 
+ and hence we will try to do regression with some independent variables to remove any significant effect

In our case, the DW test statistic is near to zero which indicates there is significant impact of time in the above data 


LSTM

In [ ]:
timeseries = df_sorted['actual_packs_order'].values

In [ ]:
scaler =  MinMaxScaler(feature_range=(0, 1))
timeseries = scaler.fit_transform(np.reshape(timeseries,(-1,1)))

#### Dividing into Train and Test

In [ ]:
train_size = int(len(timeseries) * 0.9)
test_size = len(timeseries) - train_size
train, test = timeseries[0:train_size,:], timeseries[train_size:len(timeseries),:]
print(len(train), len(test))


In [ ]:
# fix random seed for reproducibility
np.random.seed(7)

+ The function takes two arguments: the dataset, 
+ which is a NumPy array that we want to convert into a dataset, 
+ and the look_back, which is the number of previous time steps 
+ to use as input variables to predict the next time period — in this case defaulted to 1.


In [ ]:
def create_dataset(dataset, look_back):
    dataX, dataY = [], []
    for i in range(len(dataset)-look_back-1):
        a = dataset[i:(i+look_back), 0]
        dataX.append(a)
        dataY.append(dataset[i + look_back, 0])
    return np.array(dataX), np.array(dataY)

In [ ]:
look_back = 10
trainX, trainY = create_dataset(train, look_back)
testX, testY = create_dataset(test, look_back)

In [ ]:
# reshape input to be [samples, time steps, features]
trainX = np.reshape(trainX, (trainX.shape[0], 1, trainX.shape[1]))
testX = np.reshape(testX, (testX.shape[0], 1, testX.shape[1]))

In [ ]:
trainX[0]

In [ ]:
# create and fit the LSTM network
model = Sequential()
model.add(LSTM(2, input_shape=(1, look_back)))
model.add(Dense(1))

In [ ]:
model.compile(loss='mean_squared_error', optimizer='adam')
model.fit(trainX, trainY, epochs=100, batch_size=1, verbose=1,validation_split= 0.1)

In [ ]:
# make predictions
trainPredict = model.predict(trainX)
testPredict = model.predict(testX)

# # # invert predictions
trainPredict = scaler.inverse_transform(np.reshape(trainPredict,(-1,1)))
trainY = scaler.inverse_transform(np.reshape([trainY],(-1,1)))
testPredict = scaler.inverse_transform(np.reshape(testPredict,(-1,1)))
testY = scaler.inverse_transform(np.reshape([testY],(-1,1)))


#### Train Prediction

In [ ]:
# shift train predictions for plotting
trainPredictPlot = np.empty_like(timeseries)
trainPredictPlot[:, :] = np.nan
trainPredictPlot[look_back:len(trainPredict)+look_back, :] = trainPredict

# shift test predictions for plotting
testPredictPlot = np.empty_like(timeseries)
testPredictPlot[:, :] = np.nan
testPredictPlot[len(trainPredict)+(look_back*2)+1:len(timeseries)-1, :] = testPredict

# plot baseline and predictions
plt.plot(scaler.inverse_transform(timeseries))
plt.plot(trainPredictPlot)
#plt.xlim(600,690)

In [ ]:
plt.plot(scaler.inverse_transform(timeseries))
plt.plot(testPredictPlot)
plt.show()